### COURSERA | IBM DATA SCIENCE | LOAN CLASSIFIER
The purpose of this program is to construct various models to predict whether a customer will pay off or default on a loan based on several factors such as age, education, etc. <br> Models include K-Nearest Neighbors (finding optimal K), Decision Tree, Support Vector Machine, and Logistic Regression. <br> Model evaluation included accuracy, F1 Score, Jaccard Index, and Log Loss when applicable <br><br>
Data were split into training and testing which were respectively used to train and test models. <br> This program uses loan data provided by Coursera.

#### PART 1: PREPARE & CLEAN LOAN DATA

In [1]:
import pandas as pd
import numpy as np

In [2]:
!wget -O loan_train.csv https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0101ENv3/labs/loan_train.csv
print('SUCCESSFULLY DOWNLOADED FILE')

--2019-08-14 23:10:06--  https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0101ENv3/labs/loan_train.csv
Resolving s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)... 67.228.254.193
Connecting to s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)|67.228.254.193|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23101 (23K) [text/csv]
Saving to: ‘loan_train.csv’

100%[======================================>] 23,101      --.-K/s   in 0.002s  

2019-08-14 23:10:06 (12.6 MB/s) - ‘loan_train.csv’ saved [23101/23101]

SUCCESSFULLY DOWNLOADED FILE


The __loan_train.csv__ dataset includes data regarding 346 customers whose loan was either paid off or defaulted. Attributes include the following:

| Field          | Description                                                                           |
|----------------|---------------------------------------------------------------------------------------|
| Loan Status    | Loan is either paid off or in collection                                              |
| Principal      | Principal Loan Amount                                                                 |
| Terms          | Weekly, Biweekly, or Monthly Payoff Schedule                                          |
| Effective Date | Loan Date                                                                             |
| Due Date       | One-time Payoff Date                                                                  |
| Age            | Age of Applicant                                                                      |
| Education      | Education of Applicant                                                                |
| Gender         | Gender of Applicant                                                                   |

In [3]:
df = pd.read_csv('loan_train.csv')
df.drop(columns=['Unnamed: 0', 'Unnamed: 0.1'], inplace=True)
df.columns = ['Loan Status', 'Principal', 'Terms', 'Effective Date', 'Due Date', 'Age', 'Education', 'Gender']
df['Education'].replace('Bechalor', 'Bachelor', inplace=True)
df['Education'].replace('college', 'College', inplace=True)
df['Gender'].replace('male', 'M', inplace=True)
df['Gender'].replace('female', 'F', inplace=True)
df.head()

,Loan Status,Principal,Terms,Effective Date,Due Date,Age,Education,Gender
0,PAIDOFF,1000,30,9/8/2016,10/7/2016,45,High School or Below,M
1,PAIDOFF,1000,30,9/8/2016,10/7/2016,33,Bachelor,F
2,PAIDOFF,1000,15,9/8/2016,9/22/2016,27,College,M
3,PAIDOFF,1000,30,9/9/2016,10/8/2016,28,College,F
4,PAIDOFF,1000,30,9/9/2016,10/8/2016,29,College,M


In [4]:
print('ENTRIES IN DATAFRAME:', len(df))

ENTRIES IN DATAFRAME: 346


In [5]:
df['Due Date'] = pd.to_datetime(df['Due Date'])
df['Effective Date'] = pd.to_datetime(df['Effective Date'])
df.head()

,Loan Status,Principal,Terms,Effective Date,Due Date,Age,Education,Gender
0,PAIDOFF,1000,30,2016-09-08,2016-10-07,45,High School or Below,M
1,PAIDOFF,1000,30,2016-09-08,2016-10-07,33,Bachelor,F
2,PAIDOFF,1000,15,2016-09-08,2016-09-22,27,College,M
3,PAIDOFF,1000,30,2016-09-09,2016-10-08,28,College,F
4,PAIDOFF,1000,30,2016-09-09,2016-10-08,29,College,M


In [6]:
df['Status'].value_counts()

KeyError: 'Status'

#### PART 2: FEATURE SELECTION & EXTRACTION

In [ ]:
import itertools
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from matplotlib.ticker import NullFormatter
from sklearn import preprocessing
%matplotlib inline

In [ ]:
bins = np.linspace(df.Principal.min(), df.Principal.max(), 10)
g = sns.FacetGrid(df, col="Gender", hue="Loan Status", palette="Set1", col_wrap=2)
g.map(plt.hist, 'Principal', bins=bins, ec="k")
g.axes[-1].legend()
plt.show()

In [ ]:
bins = np.linspace(df.Age.min(), df.Age.max(), 10)
g = sns.FacetGrid(df, col="Gender", hue="Loan Status", palette="Set1", col_wrap=2)
g.map(plt.hist, 'Age', bins=bins, ec="k")
g.axes[-1].legend()
plt.show()

In [ ]:
df['Day'] = df['Effective Date'].dt.dayofweek
bins = np.linspace(df.Day.min(), df.Day.max(), 10)
g = sns.FacetGrid(df, col="Gender", hue="Loan Status", palette="Set1", col_wrap=2)
g.map(plt.hist, 'Day', bins=bins, ec="k")
g.axes[-1].legend()
plt.show()

#### PART 3: ONEHOT ENCODE, NORMALIZE, & SPLIT DATA

In [ ]:
df['Weekend'] = df['Day'].apply(lambda x: 1 if (x>3) else 0)
df.head()

In [ ]:
df.groupby(['Gender'])['Loan Status'].value_counts(normalize=True)

In [ ]:
df['Gender'].replace(to_replace=['M','F'], value=[0,1], inplace=True)
df.head()

In [ ]:
df.groupby(['Education'])['Loan Status'].value_counts(normalize=True)

In [ ]:
features_df = df[['Principal', 'Terms', 'Age', 'Gender', 'Weekend']]
features_df = pd.concat([features_df, pd.get_dummies(df['Education'])], axis=1)
features_df.drop(['Master or Above'], axis = 1, inplace=True)
features_df.head()

In [ ]:
X = features_df
Y = df['Loan Status'].values
X = preprocessing.StandardScaler().fit(X).transform(X)

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=4)
print ('Training Set:', len(x_train))
print ('Test Set:', len(x_test))

#### PART 4A: CONSTRUCT K-NEAREST NEIGHBOR MODEL

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics

In [ ]:
kmax = 10
mean_acc = np.zeros(kmax-1)
std_acc = np.zeros(kmax-1)
for n in range(1, kmax):
    neigh = KNeighborsClassifier(n_neighbors = n).fit(x_train, y_train)
    yhat = neigh.predict(x_test)
    mean_acc[n-1] = metrics.accuracy_score(y_test, yhat)
    std_acc[n-1]= np.std(yhat==y_test)/np.sqrt(yhat.shape[0])
    
plt.plot(range(1, kmax), mean_acc, 'g')
plt.fill_between(range(1, kmax), mean_acc-1*std_acc, mean_acc+1*std_acc, alpha=0.1)
plt.legend('Accuracy', '+/- 3 STD')
plt.ylabel('Accuracy')
plt.xlabel('Number of Neighbors (K)')
plt.tight_layout()
plt.show()

In [ ]:
kopt = 7
neigh = KNeighborsClassifier(n_neighbors = kopt).fit(x_train, y_train)
yhat = neigh.predict(x_test)
kopt_acc = metrics.accuracy_score(y_test, yhat)
print('Optimal K:', kopt)
print('Accuracy:', kopt_acc)

#### PART 4B: CONSTRUCT DECISION TREE MODEL

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
tree = DecisionTreeClassifier(criterion="entropy", max_depth = 4).fit(x_train, y_train)
yhat = tree.predict(x_test)
tree_acc = metrics.accuracy_score(y_test, yhat)
print('Accuracy:', tree_acc)

#### PART 4C: CONSTRUCT SUPPORT VECTOR MACHINE MODEL

In [ ]:
from sklearn import svm

In [ ]:
clf = svm.SVC(kernel='rbf')
clf.fit(x_train, y_train)
yhat = clf.predict(x_test)
clf_acc = metrics.accuracy_score(y_test, yhat)
print('Accuracy:', clf_acc)

#### PART 4D: CONSTRUCT LOGISTIC REGRESSION MODEL

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix

In [ ]:
logreg = LogisticRegression(C=0.01, solver='liblinear').fit(x_train, y_train)
yhat = logreg.predict(x_test)
lr_acc = metrics.accuracy_score(y_test, yhat)
print('Accuracy:', lr_acc)

#### PART 5: MODEL EVALUATION

In [ ]:
from sklearn.metrics import jaccard_similarity_score
from sklearn.metrics import f1_score
from sklearn.metrics import log_loss

In [ ]:
!wget -O loan_test.csv https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0101ENv3/labs/loan_test.csv
print('SUCCESSFULLY DOWNLOADED FILE')

In [ ]:
df_test = pd.read_csv('loan_test.csv')
df_test.drop(columns=['Unnamed: 0', 'Unnamed: 0.1'], inplace=True)
df_test.columns = ['Loan Status', 'Principal', 'Terms', 'Effective Date', 'Due Date', 'Age', 'Education', 'Gender']
df_test['Education'].replace('Bechalor', 'Bachelor', inplace=True)
df_test['Education'].replace('college', 'College', inplace=True)
df_test['Gender'].replace('male', 'M', inplace=True)
df_test['Gender'].replace('female', 'F', inplace=True)

df_test['Due Date'] = pd.to_datetime(df_test['Due Date'])
df_test['Effective Date'] = pd.to_datetime(df_test['Effective Date'])
df_test['Day'] = df_test['Effective Date'].dt.dayofweek
df_test['Weekend'] = df_test['Day'].apply(lambda x: 1 if (x>3) else 0)
df_test['Gender'].replace(to_replace=['M','F'], value=[0,1], inplace=True)
df_test.head()

In [ ]:
x_test = df_test[['Principal', 'Terms', 'Age', 'Gender', 'Weekend']]
x_test = pd.concat([x_test, pd.get_dummies(df_test['Education'])], axis=1)
x_test.drop(['Master or Above'], axis=1, inplace=True)

y_test = df_test['Loan Status'].values
x_test = preprocessing.StandardScaler().fit(x_test).transform(x_test)

In [ ]:
yhat_test = neigh.predict(x_test)
knn_acc = metrics.accuracy_score(y_test, yhat_test)
knn_f1 = f1_score(y_test, yhat_test, average='weighted')
knn_jc = jaccard_similarity_score(y_test, yhat_test)
print('Optimal K:', kopt)
print('Accuracy:', knn_acc)
print('F1 Score:', knn_f1)
print('Jaccard Index:', knn_jc)

In [ ]:
yhat_test = tree.predict(x_test)
tree_acc = metrics.accuracy_score(y_test, yhat_test)
tree_f1 = f1_score(y_test, yhat_test, average='weighted')
tree_jc = jaccard_similarity_score(y_test, yhat_test)
print('Accuracy:', tree_acc)
print('F1 Score:', tree_f1)
print('Jaccard Index:', tree_jc)

In [ ]:
yhat_test = clf.predict(x_test)
clf_acc = metrics.accuracy_score(y_test, yhat_test)
clf_f1 = f1_score(y_test, yhat_test, average='weighted')
clf_jc = jaccard_similarity_score(y_test, yhat_test)
print('Accuracy:', clf_acc)
print('F1 Score:', clf_f1)
print('Jaccard Index:', clf_jc)

In [ ]:
yhat_test = logreg.predict(x_test)
lr_acc = metrics.accuracy_score(y_test, yhat_test)
lr_f1 = f1_score(y_test, yhat_test, average='weighted')
lr_jc = jaccard_similarity_score(y_test, yhat_test)
yhat_prob = logreg.predict_proba(x_test)
lr_ll = log_loss(y_test, yhat_prob)
print('Accuracy:', lr_acc)
print('F1 Score:', lr_f1)
print('Jaccard Index:', lr_jc)
print('Log Loss:', lr_ll)

| Algorithm           | Jaccard Index| F1 Score | Log Loss |
|---------------------|--------------|----------|----------|
| KNN                 | 0.667        | 0.633    | N/A      |
| Decision Tree       | 0.722        | 0.737    | N/A      |
| SVM                 | 0.796        | 0.758    | N/A      |
| Logistic Regression | 0.741        | 0.660    | 0.567    |